In [ ]:
# extensions
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextensions_configurator enable --user


In [5]:
import os
os.mkdir("files")

In [15]:
!pip install -q pypdf
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers
!pip install -q llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.5 MB/s eta 0:00:00


In [16]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 90.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 120.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
Failed to build llama-cpp-python
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (llama-cpp-python)


In [18]:
%pip install llama-index-embeddings-huggingface -q
%pip install llama-index-llms-llama-cpp -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 19.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.10.17 requires llama-index-core<0.11.0,>=0.10.17, but you have llama-index-core 0.11.3 which is incompatible.
llama-index-agent-openai 0.1.7 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.3 which is incompatible.
llama-index-cli 0.1.13 requires llama-index-core<0.11.0,>=0.10.11.post1, but you have llama-index-core 0.11.3 which is incompatible.
llama-index-embeddings-openai 0.1.11 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.3 which is incompatible.
llama-index-indices-managed-llama-cloud 0.1.6 requires llama-index-core<0.11.0,>=0.10.0, but you have llama-index-core 0.11.3 which is incompatible.
llama-index-llms-openai 0.1.31 requires llama-index-core<0.11.0,

In [19]:
import torch
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)


llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=4096,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_path" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


total size (MB): 4368.44


4167it [00:38, 108.02it/s]                          


In [20]:
from llama_index.core import Document

documents = SimpleDirectoryReader("/content/files", ).load_data()
documents = Document(text ="/n/n".join([doc.text for doc in documents]) )



In [40]:
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.core.storage.storage_context import StorageContext


def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):

  node_parser = SentenceWindowNodeParser(
      window_size=sentence_window_size,
      window_metadata_key="window",
      original_text_metadata_key="original_text",
  )
  Settings.llm=llm
  Settings.embed_model=embed_model
  Settings.node_parser=node_parser


  if not os.path.exists(save_dir):
    index = VectorStoreIndex.from_documents([documents], embed_model=embed_model)
    index.storage_context.persist(persist_dir=save_dir)

  else :
    index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=save_dir)
    )

  return index

In [41]:
vector_index = get_build_index(documents,llm)


In [42]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank

def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):

  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )

  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [43]:
query_engine = get_query_engine(vector_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
print("hello")